 # WHERE TO EAT???

## 1.INTRODUCTION

### 1.1 BACKGROUND

Jeff Garden, a pastry chef by profession was having a successful café in Long Island, New York for the last 5 years. His business over there has been running smoothly and he was maintaining it as a family business with his wife.

Unfortunately, the Covid 19 pandemic rushed to the country and the whole USA, most affectively New York hit very hard in every possible way, including the economy.

So Jeff ‘s café was also had to close and because of the insecurity in the city, he has planned to change the state and move to Maryland state to live and open a café over there. Since his relatives live in Rockville, Maryland his initial plan is to move to Rockville. 

### 1.2 THE PROBLEM

Jeff has been in New York all his life and he does not have any idea about Maryland and he is seeking the help of a Data Scientist to analyze his areas of his interest and seek an area which has a good potential for opening his café. His has given the Data Scientist his requirements very clearly and after analyzing them his main requirement were:

-	Jeff’s  relatives are living in Rockville, so he prefers to live around Rockville	

-	He needs the Data Scientist to analyze the areas to look for a best place, which will be either Rockville or a city nearby to open his café.

## 2.DATA ACQUISITION 

### 2.1 DATA SOURCES

After a study of his interested areas, it was revealed that, among other cities in the Montgomery county, Rockville and Potomac are the first two cities which can cater Jeff’s requirement. So, the objective is to explore the neighborhoods in the above mentioned cities and find the best city and location for Jeff to open his café.

Details regarding the cities can be obtained by the following pages:

https://en.wikipedia.org/wiki/Rockville,_Maryland  
https://en.wikipedia.org/wiki/Potomac,_Maryland


Scraping data from these websites and using it with the foursquare location data can be used to analyze and explore the cities and help Jeff in finding the best place ever for his café.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [2]:
address = 'Rockville,MD'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Rockville are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Rockville are 39.0840054, -77.1527573.


In [3]:
df_zip=pd.read_csv ('https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/New_York&lang=en&use_labels_for_header=true&csv_separator=%3B')
df_zip.head()

,Zip;City;State;Latitude;Longitude;Timezone;Daylight savings time flag;geopoint
71937;Cove;AR;34.398483;-94.39398;-6;1;34.398483,-94.39398
72044;Edgemont;AR;35.624351;-92.16056;-6;1;35.624351,-92.16056
56171;Sherburn;MN;43.660847;-94.74357;-6;1;43.660847,-94.74357
49430;Lamont;MI;43.010337;-85.89754;-5;1;43.010337,-85.89754
52585;Richland;IA;41.194129;-91.98027;-6;1;41.194129,-91.98027


In [14]:
map_rockville = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Zip']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_rockville

KeyError: 'Latitude'

Georgraphical coordinates of Rockville

In [16]:
address = 'Rockville,MD'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Rockville are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Rockville are 39.0840054, -77.1527573.


In [4]:
CLIENT_ID = 'TRYKGN22ENGVHFHDAUPDY3LPCOACQOB3IABAFQSKFWCAHYNZ' # your Foursquare ID
CLIENT_SECRET = 'CE5CDDUCH35BVFO1UYYTYOFN4UVRSO0E2Z1VFD02TT2P3XPI' # your Foursquare Secret
VERSION = '20200602'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TRYKGN22ENGVHFHDAUPDY3LPCOACQOB3IABAFQSKFWCAHYNZ
CLIENT_SECRET:CE5CDDUCH35BVFO1UYYTYOFN4UVRSO0E2Z1VFD02TT2P3XPI


Finding cafes around each zip code in rockville

In [5]:
search_query = 'cafe'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

cafe .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=TRYKGN22ENGVHFHDAUPDY3LPCOACQOB3IABAFQSKFWCAHYNZ&client_secret=CE5CDDUCH35BVFO1UYYTYOFN4UVRSO0E2Z1VFD02TT2P3XPI&ll=39.0840054,-77.1527573&v=20200602&query=cafe&radius=500&limit=30'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eec2387949393001bb70394'},
 'response': {'venues': [{'id': '4ad74d19f964a520530921e3',
    'name': 'Lebanese Taverna Café',
    'location': {'address': '115 Gibbs St Unit A',
     'crossStreet': 'btwn E. Middle Ln. & Beall St.',
     'lat': 39.0858230481144,
     'lng': -77.1519049215222,
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.0858230481144,
       'lng': -77.1519049215222},
      {'label': 'entrance', 'lat': 39.085612, 'lng': -77.151898}],
     'distance': 215,
     'postalCode': '20850',
     'cc': 'US',
     'city': 'Rockville',
     'state': 'MD',
     'country': 'United States',
     'formattedAddress': ['115 Gibbs St Unit A (btwn E. Middle Ln. & Beall St.)',
      'Rockville, MD 20850',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d115941735',
      'name': 'Middle Eastern Restaurant',
      'pluralName': 'Middle Eastern Restaurants',
      'shortName': 'Middle Eastern',
      'icon': {'prefix': 'ht

Listing all cafes to a Dataframe

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d115941735', 'name': 'M...",348867,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/lebanese-ta...,False,4ad74d19f964a520530921e3,115 Gibbs St Unit A,US,Rockville,United States,btwn E. Middle Ln. & Beall St.,215,[115 Gibbs St Unit A (btwn E. Middle Ln. & Bea...,"[{'label': 'display', 'lat': 39.0858230481144,...",39.085823,-77.151905,20850,MD,Lebanese Taverna Café,v-1592533918,NaN
1,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ae09e6af964a520fd8021e3,20 Courthouse Sq,US,Rockville,United States,NaN,44,"[20 Courthouse Sq, Rockville, MD 20850, United...","[{'label': 'display', 'lat': 39.08435, 'lng': ...",39.084350,-77.152495,20850,MD,Upon Sisters Cafe,v-1592533918,NaN
2,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",NaN,NaN,NaN,NaN,NaN,NaN,False,5316bd7f11d2c98505e2a183,12 N Washington St,US,Rockville,United States,NaN,95,"[12 N Washington St, Rockville, MD 20850, Unit...","[{'label': 'display', 'lat': 39.08467483520508...",39.084675,-77.153442,20850,MD,V Cafe,v-1592533918,NaN
3,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3225ed19836c91c7bae368,20 Courthouse Sq,US,Rockville,United States,NaN,103,"[20 Courthouse Sq, Rockville, MD 20850, United...","[{'label': 'display', 'lat': 39.08451843261719...",39.084518,-77.151756,20850,MD,Three Sisters Cafe,v-1592533918,NaN
4,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,5964068ef0b490652790d62a,101 Gibbs St,US,Rockville,United States,NaN,157,"[101 Gibbs St, Rockville, MD 20850, United Sta...","[{'label': 'display', 'lat': 39.08520582293126...",39.085206,-77.151791,20850,MD,Lab Cafe,v-1592533918,NaN


All cafes around Rockville are displayed in a map

In [8]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Rockville city',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Rockville city',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)
venues_map



Geographical coordinates of Potomac

In [9]:
address = 'Potomac,MD'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude1 = location.latitude
longitude1 = location.longitude
print('The geograpical coordinate of Potomac are {}, {}.'.format(latitude1, longitude1))

The geograpical coordinate of Potomac are 39.017936, -77.2094542.


In [10]:
search_query = 'cafe'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, search_query, radius, LIMIT)
url

cafe .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=TRYKGN22ENGVHFHDAUPDY3LPCOACQOB3IABAFQSKFWCAHYNZ&client_secret=CE5CDDUCH35BVFO1UYYTYOFN4UVRSO0E2Z1VFD02TT2P3XPI&ll=39.017936,-77.2094542&v=20200602&query=cafe&radius=500&limit=30'

Cafes around Potomac

In [11]:
results2 = requests.get(url).json()
results2

{'meta': {'code': 200, 'requestId': '5eec22901d67cb001b6ed15b'},
 'response': {'venues': [{'id': '4a8d7268f964a520aa0f20e3',
    'name': 'Vie de France Bakery Cafe- Potomac Village, MD',
    'location': {'address': '10146 River Rd',
     'crossStreet': 'at Falls Rd.',
     'lat': 39.017078829112045,
     'lng': -77.20959484870639,
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.017078829112045,
       'lng': -77.20959484870639},
      {'label': 'entrance', 'lat': 39.016822, 'lng': -77.208249}],
     'distance': 96,
     'postalCode': '20854',
     'cc': 'US',
     'city': 'Potomac',
     'state': 'MD',
     'country': 'United States',
     'formattedAddress': ['10146 River Rd (at Falls Rd.)',
      'Potomac, MD 20854',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d16a941735',
      'name': 'Bakery',
      'pluralName': 'Bakeries',
      'shortName': 'Bakery',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
       'suffi

Arrange all cafes around Potomac in to a Dataframe

In [12]:
# assign relevant part of JSON to venues
venues2 = results['response']['venues']

# tranform venues into a dataframe
dataframe2 = json_normalize(venues2)
dataframe2.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d115941735', 'name': 'M...",348867,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/lebanese-ta...,False,4ad74d19f964a520530921e3,115 Gibbs St Unit A,US,Rockville,United States,btwn E. Middle Ln. & Beall St.,215,[115 Gibbs St Unit A (btwn E. Middle Ln. & Bea...,"[{'label': 'display', 'lat': 39.0858230481144,...",39.085823,-77.151905,20850,MD,Lebanese Taverna Café,v-1592533918,NaN
1,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ae09e6af964a520fd8021e3,20 Courthouse Sq,US,Rockville,United States,NaN,44,"[20 Courthouse Sq, Rockville, MD 20850, United...","[{'label': 'display', 'lat': 39.08435, 'lng': ...",39.084350,-77.152495,20850,MD,Upon Sisters Cafe,v-1592533918,NaN
2,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",NaN,NaN,NaN,NaN,NaN,NaN,False,5316bd7f11d2c98505e2a183,12 N Washington St,US,Rockville,United States,NaN,95,"[12 N Washington St, Rockville, MD 20850, Unit...","[{'label': 'display', 'lat': 39.08467483520508...",39.084675,-77.153442,20850,MD,V Cafe,v-1592533918,NaN
3,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3225ed19836c91c7bae368,20 Courthouse Sq,US,Rockville,United States,NaN,103,"[20 Courthouse Sq, Rockville, MD 20850, United...","[{'label': 'display', 'lat': 39.08451843261719...",39.084518,-77.151756,20850,MD,Three Sisters Cafe,v-1592533918,NaN
4,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,5964068ef0b490652790d62a,101 Gibbs St,US,Rockville,United States,NaN,157,"[101 Gibbs St, Rockville, MD 20850, United Sta...","[{'label': 'display', 'lat': 39.08520582293126...",39.085206,-77.151791,20850,MD,Lab Cafe,v-1592533918,NaN


Viisualize all cafes around Potomac in the map

In [13]:
venues_map2 = folium.Map(location=[latitude1, longitude1], zoom_start=13)
folium.features.CircleMarker(
    [latitude1, longitude1],
    radius=10,
    color='red',
    popup='Potomac city',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map2)
folium.features.CircleMarker(
    [latitude1, longitude1],
    radius=10,
    color='red',
    popup='Potomac city',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map2)
venues_map2